In [24]:
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
import numpy as np
import random
from qiskit.circuit.library import QAOAAnsatz
from qiskit.visualization import plot_distribution
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from scipy.optimize import minimize
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer

In [30]:
N = 2 # Number of doctors
M = 3 # Number of patients
W = [random.randint(5,30) for _ in range(M)]
k = 1000 # penalty parameter
i = 0 # dont touch

In [48]:
model = Model()
decision = model.binary_var_matrix(N, M, name="x")

subsums = [model.sum(decision[i,l]*W[l] for l in range(M)) for i in range(N)]
main_obj = model.max(subsums)

i = 0

st_kara = (k/N)*model.sum((1-model.sum(decision[i,j] for i in range(N)))**2 for j in range(M))


model.minimize(main_obj+st_kara)
print(model.export_as_lp_string())
model = from_docplex_mp(model)
print(model)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model28

Minimize
 obj: - 1000 x_0_0 - 1000 x_0_1 - 1000 x_0_2 - 1000 x_1_0 - 1000 x_1_1
      - 1000 x_1_2 + x9 + [ 1000 x_0_0^2 + 2000 x_0_0*x_1_0 + 1000 x_0_1^2
      + 2000 x_0_1*x_1_1 + 1000 x_0_2^2 + 2000 x_0_2*x_1_2 + 1000 x_1_0^2
      + 1000 x_1_1^2 + 1000 x_1_2^2 ]/2 + 1500
Subject To
 c1: x7 - 17 x_0_0 - 15 x_0_1 - 15 x_0_2 = 0
 c2: x8 - 17 x_1_0 - 15 x_1_1 - 15 x_1_2 = 0
 c3: x9 - x7 >= 0
 c4: x9 - x8 >= 0
 c5: x10 + x11 = 1
 lc1: x10 = 1 -> x9 - x7 <= 0
 lc2: x11 = 1 -> x9 - x8 <= 0

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
       x7 Free
       x8 Free
       x9 Free
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x10 x11
End


KeyError: 'x8'

# CPLEX 

In [27]:
cplex_result = CplexOptimizer().solve(model)
gurobi_result = GurobiOptimizer().solve(model)

print("cplex")
print(cplex_result.prettyprint())


AttributeError: 'Model' object has no attribute 'variables'

# GUROBI

In [ ]:
print("gurobi")
print(gurobi_result.prettyprint())

# QUANTUM

In [ ]:
model = from_docplex_mp(model)
hamiltonian, offset = model.to_ising()

ansatz = QAOAAnsatz(hamiltonian, reps=2)